In [1]:
import cv2
import numpy as np
# Carregar os arquivos do modelo YOLO (os arquivos que nós baixamos)
net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')
# Carregar as classes - e transformando em uma lista
classes = []
f = open('coco.names', 'r')
lines = f.readlines()
f.close()

for line in lines:
    classes.append(line.strip())
# Configurar cores aleatórias para cada classe
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [2]:
# Inicializar a captura de vídeo da webcam
cap = cv2.VideoCapture(0)

In [3]:
while True:
    # Ler o próximo quadro da captura de vídeo
    ret, frame = cap.read()

    # Verificar se a captura de vídeo foi bem-sucedida
    if not ret:
        break

    # Obter as dimensões do quadro
    height, width, _ = frame.shape

    # Construir um blob a partir do quadro para o modelo YOLO
    blob = cv2.dnn.blobFromImage(frame, 1/255, (416, 416), swapRB=True, crop=False)

    # Definir as camadas de saída
    output_names = net.getUnconnectedOutLayersNames()
    output_layers = [net.getLayer(name) for name in output_names]
    output_layers = [layer.name for layer in output_layers]

    # Passar o blob pela rede neural
    net.setInput(blob)
    outputs = net.forward(output_layers)

    # Configurar as listas de detecção
    class_ids = []
    confidences = []
    boxes = []

    # Processar as saídas da rede neural
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                # Obter as coordenadas do objeto detectado
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Calcular os cantos do retângulo delimitador
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Adicionar as informações à lista de detecção
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    # Aplicar a supressão não máxima para eliminar detecções redundantes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

    # Desenhar os retângulos delimitadores e exibir as classes nas detecções
    for i in indices:
        box = boxes[i]
        x, y, w, h = box
        label = f'{classes[class_ids[i]]}: {confidences[i]:.2f}'
        color = colors[class_ids[i]]

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Exibir o quadro com as detecções
    cv2.imshow('Object Detection', frame)

    # Parar o loop se a tecla 'q' for pressionada
    if cv2.waitKey(1) == ord('q'):
        break

# Liberar os recursos
cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 